In [1]:
import cv2
import numpy as np
from importnb import Notebook
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy.fft as fft
import numpy.matlib
from skimage.transform import rotate
import sys
from tqdm import tqdm_notebook as tqdm

In [2]:
with Notebook(): 
        import utils
        import pinkNoise

In [3]:
def generateKernels(r_max, inc_angle, patch_size, r_min=5):
    """
    @param r_max: maximum kernel size
    @param inc_angle: incremental angle
    @param patch_size: patch size of kernels
    @param r_min: minimum kernel size
    
    @return out: matrix containg kernels of size r_min to r_max with angles incremented with inc_angle
    
    """
    if r_min % 2==0:
        r_min += 1
        
    out = []
    for mag in range(r_max, r_max+1, 2):
        for ang in range(0, 180, inc_angle):
            kernel = utils.blurKernel(mag, ang)
            pd = (patch_size- kernel.shape[0])//2
            kernel = np.pad(kernel, ((pd, patch_size- kernel.shape[0]-pd), (pd, patch_size- kernel.shape[0]-pd)), 'constant', constant_values=(0, 0))
            out.append(kernel.ravel())
    
    print(np.array(out).shape)
    return np.array(out).T ##shape  patch_size x mag*ang

In [4]:
def getImageBlurEstimates(img, oimg, kernels, patch_size):
    """
    @param img: input image
    @param oimg:
    @param kernels: kernels amongst which blur kernel needs to be chosen based on mse loss
    @param patch_size: patch size to consider in estimation
    
    @return kidxs: blur estimates for the image
    """
    
    pd = patch_size//2
    _,nimg = pinkNoise.addNoise(img, -1)
    nimg = np.pad(nimg, ((pd,pd), (pd,pd)), 'reflect')
    oimg = np.pad(img, ((pd,pd), (pd,pd)), 'reflect')
    
 
    cimg = utils.im2col(oimg, [patch_size, patch_size])
    cnimg = utils.im2col(nimg, [patch_size, patch_size])
    kidxs = []
    for k in tqdm(range(kernels.shape[1])):
        fi = np.sum(kernels[:,k][:,None] * cnimg, axis=0).reshape(img.shape)
        fi = np.pad(fi, ((pd,pd), (pd,pd)), 'reflect')
        cfi = utils.im2col(fi, [patch_size, patch_size])
        
        kidxs.append(np.sum((cfi - cimg)**2,axis=0))
        
    kidxs = np.argmin(np.array(kidxs), axis=0)
    
#     kidxs = np.argmin((cnimg.T @ kernels - cimg[patch_size**2//2, :][:,None])**2, axis = 1)
    
    del cnimg
    del cimg
    del nimg
    
    return kidxs

# TESTING

In [16]:
# img = utils.loadImage('./resources/blu.png', 'gray')
# img = utils.loadImage('./resources/building_small.jpg', 'gray')
# img = np.array(img, dtype=np.float32)
# img = utils.linearBlur(img, 11, 45)
# cv2.imwrite('blurredbuilding11-45.jpg', img)
# plt.imshow(img, cmap='gray')
# plt.savefig('blirredbuilding11-45.jpg', bbox_inches='tight')
# noise, noised_img = pinkNoise.addNoise(img, exponent = -1)
# cv2.imwrite('noisedbuilding.jpg', noised_img)
# kernels = generateKernels(14, 5, 35)
# sel_kernels = getImageBlurEstimates(img, img,kernels, 35)# _, plots = plt.subplots(1,2,figsize=(20,20))
# sel_kernels = getImageBlurEstimate(img, 35)# _, plots = plt.subplots(1,2,figsize=(20,20))

# print(sel_kernels.shape)
# # plots[0].imshow(noise, cmap = 'gray')
# # plots[1].imshow(noised_img, cmap = 'gray')
# # plt.show()
# print(sel_kernels[:,1500].reshape(21,21))


image ./resources/building_small.jpg loaded
(36, 1225)
28.792639060985834


In [18]:
# print(np.unique(sel_kernels, return_counts=True))
# print(np.sum(sel_kernes==114))
# kernels[:, sel_kernels][:,-1].reshape(21,21)
# x, y = np.unique(sel_kernels, return_counts=True)
# for i, ind in enumerate(x):
#     print('angle =',i*5, 'degrees --> count =', y[ind])
    


(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35]), array([ 281,  790,  331,  873, 1138, 1886, 2627, 3101, 5911, 8084, 3614,
       3964, 3532, 2488, 2310, 4990, 6066, 6682, 1765, 1568,  135,  205,
        174, 1915,  294,  542,  422,  191, 3210,  315,  343, 2076,  956,
       1640, 1169, 6312]))
angle = 0 degrees --> count = 281
angle = 5 degrees --> count = 790
angle = 10 degrees --> count = 331
angle = 15 degrees --> count = 873
angle = 20 degrees --> count = 1138
angle = 25 degrees --> count = 1886
angle = 30 degrees --> count = 2627
angle = 35 degrees --> count = 3101
angle = 40 degrees --> count = 5911
angle = 45 degrees --> count = 8084
angle = 50 degrees --> count = 3614
angle = 55 degrees --> count = 3964
angle = 60 degrees --> count = 3532
angle = 65 degrees --> count = 2488
angle = 70 degrees --> count = 2310
angle = 75 degrees --> count = 4990
angle = 80 degre

In [8]:
# magnitude, directions = getImageBlurEstimate(img[200:235,200:235])
# # magnitude, directions = findPatchKernel(img[100:100,200:350])
# print(magnitude, directions)

In [9]:
# blurLegend(directions, magnitude)
# np.set_printoptions(threshold=sys.maxsize)
# print((directions==45).sum()/(150*150))

In [19]:
A=np.random.uniform(size=(10, 9))
B=np.random.uniform(size=(9,100))

c = np.einsum('ij,jk->ijk', A,B)
np.argmin(np.sum(np.abs(c - B.reshape((1,9,100))), axis=1), axis=0)

array([1, 6, 1, 6, 6, 6, 6, 1, 6, 1, 8, 6, 1, 6, 8, 6, 1, 6, 6, 4, 1, 6,
       1, 6, 4, 1, 6, 1, 6, 4, 6, 6, 6, 6, 8, 1, 4, 6, 1, 6, 8, 1, 6, 6,
       4, 6, 6, 6, 6, 4, 6, 6, 1, 1, 6, 4, 1, 6, 4, 6, 6, 4, 8, 6, 1, 1,
       4, 6, 6, 4, 6, 4, 1, 4, 4, 6, 6, 6, 6, 4, 8, 1, 6, 1, 6, 6, 1, 4,
       8, 6, 6, 6, 1, 1, 8, 6, 6, 8, 6, 6])